# Syn vs Nonsyn and Non-nonsense vs Nonsense mutation rates

## Based on positions, instead of codons!

This makes it easier to use these for FDR estimation.

In [8]:
%run "Header.ipynb"
%run "GeneUtils.ipynb"

In [9]:
import json
import pickle
import skbio
import pileup
from statistics import median
from math import log
from collections import defaultdict
from matplotlib import pyplot
from parse_sco import parse_sco

## Percentages (values of $p$) for naive mutation calling

### Values we use for plotting
Currently the same as in the codon mutation notebook. I guess these should be a shared variable, ideally? But that might be too much work for its own good.

In [10]:
percentages = [2, 1, 0.5, 0.25, 0.15]

### Full values, used for plotting a FDR curve elsewhere

Copied from the target/decoy notebook. may be nice to generalize to shared code too i guess

In [11]:
# Percentages go from 4.99%, 4.98%, ..., 0.16%, 0.15%
full_percentages = [p / 100 for p in range(15, 500, 1)][::-1]
print(f"First two percentages: {full_percentages[:2]}")
print(f"Last two percentages: {full_percentages[-2:]}")
print(f"Number of percentages: {len(full_percentages):,}")

First two percentages: [4.99, 4.98]
Last two percentages: [0.16, 0.15]
Number of percentages: 485


In [12]:
# Sanity check -- we can just compute this data for all of the stuff in full_percentages
# and then for the stuff we wanna plot in this notebook just focus on the values in percentages
for p in percentages:
    assert p in full_percentages, f"{p}% not in full_percentages"

## Find _possibly nonsynonymous_ / _possibly nonsense_ positions

This analysis is assuming of course that the standard genetic code applies to the microbes we are studying. Which is eventually going to be unrealistic, but for the purposes of what we're doing here it should be ok.

In [51]:
codons = []
dna = "ACGT"
for x in dna:
    for y in dna:
        for z in dna:
            codons.append(x+y+z)
            
# Maps the 64 codons --> [1, 2, 3] --> an integer 0, 1, 2, or 3, indicating how many of the 3 mutations
# from this CP in this codon into another nucleotide are synonymous (si) or nonsynonymous (ni)
codon2cp2si = defaultdict(dict)
codon2cp2ni = defaultdict(dict)

# Like above, but for non-nonsense (nnsi) or nonsense (nsi) mutations and only for the 61 sense codons
codon2cp2nnsi = defaultdict(dict)
codon2cp2nsi = defaultdict(dict)

stop_codons = []

for c in codons:
    is_not_stop = not (str(skbio.DNA(c).translate()) == "*")
    if not is_not_stop:
        stop_codons.append(c)
    for pos in [0, 1, 2]:
        si = 0
        ni = 0
        nnsi = 0
        nsi = 0
        for alt_nt in set(dna) - set(c[pos]):
            alt_codon = c[:pos] + alt_nt + c[pos + 1:]
            aa1 = str(skbio.DNA(c).translate())
            aa2 = str(skbio.DNA(alt_codon).translate())
            if aa1 == aa2:
                si += 1
                if is_not_stop:
                    nnsi += 1
            else:
                ni += 1
                if is_not_stop:
                    if aa2 == "*":
                        nsi += 1
                    else:
                        nnsi += 1
                    
        assert si + ni == 3
        codon2cp2si[c][pos + 1] = si
        codon2cp2ni[c][pos + 1] = ni
        
        if is_not_stop:
            assert nsi + nnsi == 3
            codon2cp2nnsi[c][pos + 1] = nnsi
            codon2cp2nsi[c][pos + 1] = nsi
            
assert len(codon2cp2si) == len(codon2cp2ni) == 64
assert len(codon2cp2nnsi) == len(codon2cp2nsi) == 61
assert len(stop_codons) == 3

In [53]:
seq2num_possibly_nonsynonymous_positions = {}
seq2num_possibly_nonsense_positions = {}

for seq in SEQS:
    df = parse_sco(f"../seqs/genes/{seq}.sco")
    fasta = skbio.DNA.read(f"../seqs/{seq}.fasta")
    # Figure out quickly which positions are in multiple genes, so we can ignore them
    pos_to_genes = get_parent_gene_info_of_many_positions(df)
    
    # Nmber of positions where, if this position is mutated into at least one of the other
    # three possible nucleotides, this would trigger a [nonsynonymous/nonsense] mutation
    num_poss_nonsyn_positions = 0
    num_poss_nonsense_positions = 0
    
    for gene in df.itertuples():
        print(
            f"{seq2name[seq]}: On gene {gene.Index:,} / {len(df.index):,}. "
            f"So far {num_poss_nonsyn_positions:,} poss nonsyn and {num_poss_nonsense_positions:,} "
            f"poss nonsense positions."
        )
        # The order of positions we go through is dependent on gene orientation:
        # left --> right if this is a "+" strand gene,
        # right --> left if this is a "-" strand gene.
        pos_interval = get_pos_interval_from_gene(gene)
        
        # We'll update these values as we walk through the gene.
        # curr_codon_cp1_pos indicates CP1 (the left end for + genes, the right end for - genes)
        # of the current gene. It's 1-indexed, along with the gene coordinates.
        curr_codon_cp1_pos = pos_interval[0]
        # Always either 1, 2, or 3
        cp = 1
        
        for pos in pos_interval:
            if cp == 1:
                # It's easier to sync this at the top rather than bottom of the loop because the positions
                # in pos_interval could be increasing or decreasing depending on the gene's strand
                curr_codon_cp1_pos = pos
            
            num_parent_genes = len(pos_to_genes[pos])
            if num_parent_genes == 1:
                
                if gene.Strand == "+":
                    # The skbio sequence is 0-indexed, so we gotta subtract by 1
                    parent_codon_fasta = fasta[curr_codon_cp1_pos - 1: curr_codon_cp1_pos + 2]
                elif gene.Strand == "-":
                    parent_codon_fasta = fasta[curr_codon_cp1_pos - 3: curr_codon_cp1_pos].reverse_complement()
                else:
                    raise ValueError(f"Invalid strand: {gene.Strand}")
                
                parent_codon_str = str(parent_codon_fasta)
                if codon2cp2ni[parent_codon_str][cp] > 0:
                    num_poss_nonsyn_positions += 1
                    
                    if parent_codon_str not in stop_codons:
                        # If this position cannot be nonsynonymously mutated at all, then it cannot be possibly
                        # be nonsense-ly mutated. So we only need to try to update the number of possible
                        # nonsense-ly mutated positions if the number of possible nonsynonymously mutated
                        # positions is > 0. ..."Nonsense-ly" isn't an actual word, but nobody is reading this,
                        # so we're gonna let that slide for now.
                        if codon2cp2nsi[parent_codon_str][cp] > 0:
                            num_poss_nonsense_positions += 1
                    
            cp = next_cp(cp)
    seq2num_possibly_nonsynonymous_positions[seq] = num_poss_nonsyn_positions
    seq2num_possibly_nonsense_positions[seq] = num_poss_nonsense_positions

CAMP: On gene 1 / 1,297. So far 0 poss nonsyn and 0 poss nonsense positions.
CAMP: On gene 2 / 1,297. So far 400 poss nonsyn and 58 poss nonsense positions.
CAMP: On gene 3 / 1,297. So far 736 poss nonsyn and 113 poss nonsense positions.
CAMP: On gene 4 / 1,297. So far 1,126 poss nonsyn and 168 poss nonsense positions.
CAMP: On gene 5 / 1,297. So far 2,283 poss nonsyn and 315 poss nonsense positions.
CAMP: On gene 6 / 1,297. So far 2,516 poss nonsyn and 349 poss nonsense positions.
CAMP: On gene 7 / 1,297. So far 3,450 poss nonsyn and 472 poss nonsense positions.
CAMP: On gene 8 / 1,297. So far 3,936 poss nonsyn and 525 poss nonsense positions.
CAMP: On gene 9 / 1,297. So far 4,625 poss nonsyn and 618 poss nonsense positions.
CAMP: On gene 10 / 1,297. So far 5,076 poss nonsyn and 679 poss nonsense positions.
CAMP: On gene 11 / 1,297. So far 5,218 poss nonsyn and 701 poss nonsense positions.
CAMP: On gene 12 / 1,297. So far 6,815 poss nonsyn and 894 poss nonsense positions.
CAMP: On gen

CAMP: On gene 114 / 1,297. So far 85,266 poss nonsyn and 10,836 poss nonsense positions.
CAMP: On gene 115 / 1,297. So far 85,447 poss nonsyn and 10,860 poss nonsense positions.
CAMP: On gene 116 / 1,297. So far 86,157 poss nonsyn and 10,955 poss nonsense positions.
CAMP: On gene 117 / 1,297. So far 86,754 poss nonsyn and 11,035 poss nonsense positions.
CAMP: On gene 118 / 1,297. So far 87,552 poss nonsyn and 11,144 poss nonsense positions.
CAMP: On gene 119 / 1,297. So far 88,288 poss nonsyn and 11,242 poss nonsense positions.
CAMP: On gene 120 / 1,297. So far 89,341 poss nonsyn and 11,382 poss nonsense positions.
CAMP: On gene 121 / 1,297. So far 90,435 poss nonsyn and 11,503 poss nonsense positions.
CAMP: On gene 122 / 1,297. So far 90,742 poss nonsyn and 11,548 poss nonsense positions.
CAMP: On gene 123 / 1,297. So far 91,014 poss nonsyn and 11,586 poss nonsense positions.
CAMP: On gene 124 / 1,297. So far 91,811 poss nonsyn and 11,690 poss nonsense positions.
CAMP: On gene 125 / 1

CAMP: On gene 206 / 1,297. So far 160,171 poss nonsyn and 20,472 poss nonsense positions.
CAMP: On gene 207 / 1,297. So far 161,491 poss nonsyn and 20,613 poss nonsense positions.
CAMP: On gene 208 / 1,297. So far 162,628 poss nonsyn and 20,753 poss nonsense positions.
CAMP: On gene 209 / 1,297. So far 163,680 poss nonsyn and 20,866 poss nonsense positions.
CAMP: On gene 210 / 1,297. So far 164,857 poss nonsyn and 20,994 poss nonsense positions.
CAMP: On gene 211 / 1,297. So far 165,463 poss nonsyn and 21,065 poss nonsense positions.
CAMP: On gene 212 / 1,297. So far 166,166 poss nonsyn and 21,163 poss nonsense positions.
CAMP: On gene 213 / 1,297. So far 167,360 poss nonsyn and 21,297 poss nonsense positions.
CAMP: On gene 214 / 1,297. So far 167,997 poss nonsyn and 21,371 poss nonsense positions.
CAMP: On gene 215 / 1,297. So far 169,028 poss nonsyn and 21,507 poss nonsense positions.
CAMP: On gene 216 / 1,297. So far 170,196 poss nonsyn and 21,635 poss nonsense positions.
CAMP: On g

CAMP: On gene 313 / 1,297. So far 256,080 poss nonsyn and 32,537 poss nonsense positions.
CAMP: On gene 314 / 1,297. So far 257,154 poss nonsyn and 32,675 poss nonsense positions.
CAMP: On gene 315 / 1,297. So far 257,549 poss nonsyn and 32,730 poss nonsense positions.
CAMP: On gene 316 / 1,297. So far 259,511 poss nonsyn and 32,999 poss nonsense positions.
CAMP: On gene 317 / 1,297. So far 260,125 poss nonsyn and 33,082 poss nonsense positions.
CAMP: On gene 318 / 1,297. So far 262,040 poss nonsyn and 33,335 poss nonsense positions.
CAMP: On gene 319 / 1,297. So far 262,791 poss nonsyn and 33,421 poss nonsense positions.
CAMP: On gene 320 / 1,297. So far 263,113 poss nonsyn and 33,458 poss nonsense positions.
CAMP: On gene 321 / 1,297. So far 263,987 poss nonsyn and 33,573 poss nonsense positions.
CAMP: On gene 322 / 1,297. So far 266,068 poss nonsyn and 33,846 poss nonsense positions.
CAMP: On gene 323 / 1,297. So far 267,170 poss nonsyn and 33,995 poss nonsense positions.
CAMP: On g

CAMP: On gene 407 / 1,297. So far 341,987 poss nonsyn and 43,546 poss nonsense positions.
CAMP: On gene 408 / 1,297. So far 343,310 poss nonsyn and 43,748 poss nonsense positions.
CAMP: On gene 409 / 1,297. So far 343,856 poss nonsyn and 43,801 poss nonsense positions.
CAMP: On gene 410 / 1,297. So far 344,353 poss nonsyn and 43,871 poss nonsense positions.
CAMP: On gene 411 / 1,297. So far 345,442 poss nonsyn and 43,970 poss nonsense positions.
CAMP: On gene 412 / 1,297. So far 345,740 poss nonsyn and 44,011 poss nonsense positions.
CAMP: On gene 413 / 1,297. So far 345,963 poss nonsyn and 44,037 poss nonsense positions.
CAMP: On gene 414 / 1,297. So far 346,714 poss nonsyn and 44,115 poss nonsense positions.
CAMP: On gene 415 / 1,297. So far 346,951 poss nonsyn and 44,146 poss nonsense positions.
CAMP: On gene 416 / 1,297. So far 348,559 poss nonsyn and 44,343 poss nonsense positions.
CAMP: On gene 417 / 1,297. So far 349,585 poss nonsyn and 44,446 poss nonsense positions.
CAMP: On g

...

BACT2: On gene 2,113 / 2,567. So far 1,580,844 poss nonsyn and 187,423 poss nonsense positions.
BACT2: On gene 2,114 / 2,567. So far 1,581,303 poss nonsyn and 187,462 poss nonsense positions.
BACT2: On gene 2,115 / 2,567. So far 1,582,578 poss nonsyn and 187,613 poss nonsense positions.
BACT2: On gene 2,116 / 2,567. So far 1,583,552 poss nonsyn and 187,724 poss nonsense positions.
BACT2: On gene 2,117 / 2,567. So far 1,584,607 poss nonsyn and 187,845 poss nonsense positions.
BACT2: On gene 2,118 / 2,567. So far 1,585,332 poss nonsyn and 187,917 poss nonsense positions.
BACT2: On gene 2,119 / 2,567. So far 1,585,734 poss nonsyn and 187,974 poss nonsense positions.
BACT2: On gene 2,120 / 2,567. So far 1,587,097 poss nonsyn and 188,111 poss nonsense positions.
BACT2: On gene 2,121 / 2,567. So far 1,587,694 poss nonsyn and 188,187 poss nonsense positions.
BACT2: On gene 2,122 / 2,567. So far 1,588,200 poss nonsyn and 188,236 poss nonsense positions.
BACT2: On gene 2,123 / 2,567. So far 1,5

BACT2: On gene 2,209 / 2,567. So far 1,658,381 poss nonsyn and 196,816 poss nonsense positions.
BACT2: On gene 2,210 / 2,567. So far 1,658,542 poss nonsyn and 196,842 poss nonsense positions.
BACT2: On gene 2,211 / 2,567. So far 1,659,092 poss nonsyn and 196,906 poss nonsense positions.
BACT2: On gene 2,212 / 2,567. So far 1,659,366 poss nonsyn and 196,930 poss nonsense positions.
BACT2: On gene 2,213 / 2,567. So far 1,660,340 poss nonsyn and 197,012 poss nonsense positions.
BACT2: On gene 2,214 / 2,567. So far 1,661,457 poss nonsyn and 197,124 poss nonsense positions.
BACT2: On gene 2,215 / 2,567. So far 1,662,530 poss nonsyn and 197,242 poss nonsense positions.
BACT2: On gene 2,216 / 2,567. So far 1,663,742 poss nonsyn and 197,354 poss nonsense positions.
BACT2: On gene 2,217 / 2,567. So far 1,665,744 poss nonsyn and 197,603 poss nonsense positions.
BACT2: On gene 2,218 / 2,567. So far 1,666,135 poss nonsyn and 197,660 poss nonsense positions.
BACT2: On gene 2,219 / 2,567. So far 1,6

BACT2: On gene 2,298 / 2,567. So far 1,714,838 poss nonsyn and 203,641 poss nonsense positions.
BACT2: On gene 2,299 / 2,567. So far 1,715,322 poss nonsyn and 203,715 poss nonsense positions.
BACT2: On gene 2,300 / 2,567. So far 1,716,359 poss nonsyn and 203,860 poss nonsense positions.
BACT2: On gene 2,301 / 2,567. So far 1,717,533 poss nonsyn and 204,013 poss nonsense positions.
BACT2: On gene 2,302 / 2,567. So far 1,718,574 poss nonsyn and 204,140 poss nonsense positions.
BACT2: On gene 2,303 / 2,567. So far 1,719,116 poss nonsyn and 204,204 poss nonsense positions.
BACT2: On gene 2,304 / 2,567. So far 1,719,360 poss nonsyn and 204,231 poss nonsense positions.
BACT2: On gene 2,305 / 2,567. So far 1,720,058 poss nonsyn and 204,338 poss nonsense positions.
BACT2: On gene 2,306 / 2,567. So far 1,721,136 poss nonsyn and 204,492 poss nonsense positions.
BACT2: On gene 2,307 / 2,567. So far 1,721,860 poss nonsyn and 204,583 poss nonsense positions.
BACT2: On gene 2,308 / 2,567. So far 1,7

BACT2: On gene 2,387 / 2,567. So far 1,767,418 poss nonsyn and 210,604 poss nonsense positions.
BACT2: On gene 2,388 / 2,567. So far 1,767,584 poss nonsyn and 210,621 poss nonsense positions.
BACT2: On gene 2,389 / 2,567. So far 1,767,914 poss nonsyn and 210,657 poss nonsense positions.
BACT2: On gene 2,390 / 2,567. So far 1,768,034 poss nonsyn and 210,673 poss nonsense positions.
BACT2: On gene 2,391 / 2,567. So far 1,768,475 poss nonsyn and 210,719 poss nonsense positions.
BACT2: On gene 2,392 / 2,567. So far 1,769,004 poss nonsyn and 210,773 poss nonsense positions.
BACT2: On gene 2,393 / 2,567. So far 1,769,191 poss nonsyn and 210,799 poss nonsense positions.
BACT2: On gene 2,394 / 2,567. So far 1,769,648 poss nonsyn and 210,862 poss nonsense positions.
BACT2: On gene 2,395 / 2,567. So far 1,769,781 poss nonsyn and 210,881 poss nonsense positions.
BACT2: On gene 2,396 / 2,567. So far 1,769,954 poss nonsyn and 210,900 poss nonsense positions.
BACT2: On gene 2,397 / 2,567. So far 1,7

BACT2: On gene 2,491 / 2,567. So far 1,852,949 poss nonsyn and 220,464 poss nonsense positions.
BACT2: On gene 2,492 / 2,567. So far 1,854,169 poss nonsyn and 220,576 poss nonsense positions.
BACT2: On gene 2,493 / 2,567. So far 1,854,912 poss nonsyn and 220,678 poss nonsense positions.
BACT2: On gene 2,494 / 2,567. So far 1,855,849 poss nonsyn and 220,782 poss nonsense positions.
BACT2: On gene 2,495 / 2,567. So far 1,858,739 poss nonsyn and 221,120 poss nonsense positions.
BACT2: On gene 2,496 / 2,567. So far 1,859,044 poss nonsyn and 221,156 poss nonsense positions.
BACT2: On gene 2,497 / 2,567. So far 1,862,014 poss nonsyn and 221,504 poss nonsense positions.
BACT2: On gene 2,498 / 2,567. So far 1,863,888 poss nonsyn and 221,721 poss nonsense positions.
BACT2: On gene 2,499 / 2,567. So far 1,864,029 poss nonsyn and 221,733 poss nonsense positions.
BACT2: On gene 2,500 / 2,567. So far 1,864,771 poss nonsyn and 221,826 poss nonsense positions.
BACT2: On gene 2,501 / 2,567. So far 1,8